In [1]:
from abstract_models.imputation import RandomNumericImputer

ModuleNotFoundError: No module named 'abstract_models'

In [2]:
import os

os.getcwd()

'C:\\Users\\borut.flis\\PycharmProjects\\models\\early_diagnosis\\notebooks'

In [ ]:
a

In [3]:
import sys

for pth in sys.path:
    print(pth)

C:\Users\borut.flis\PycharmProjects\models\early_diagnosis\notebooks
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\python311.zip
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\DLLs
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\Lib
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF

C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\Lib\site-packages
c:\users\borut.flis\pycharmprojects\silicofcm
c:\users\borut.flis\pycharmprojects\stratifyhfml\stratifyhfml
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\Lib\site-packages\win32
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\Lib\site-packages\win32\lib
C:\Users\borut.flis\AppData\Local\miniconda3\envs\stratifyHF\Lib\site-packages\Pythonwin


In [ ]:
import os 

import pandas as pd

DATA_DIR = "../data"
RESULTS_DIR = os.path.join(DATA_DIR, "results")

In [ ]:
results_df = pd.concat([
    pd.read_csv(os.path.join(RESULTS_DIR, f)).assign(cv_split=f) for f in ["ID_k_fold_HFD.csv", "centre_k_fold_HFD.csv", "centre_virtual_k_fold_HFD.csv"]
], ignore_index=True)

In [ ]:
grouped_results_df = pd.concat([
    results_df.groupby(["imputer", "cv_split", "Model", "attr_group", "target"])[["AUC", "Accuracy", "Sensitivity (Recall)", "Specificity"]].mean(),
    results_df.groupby(["imputer", "cv_split", "Model", "attr_group", "target"])[["n_positive", "n_total"]].sum()
], axis=1).reset_index().drop("imputer", axis=1)

In [ ]:
grouped_results_df["target"] = grouped_results_df["target"].str.lstrip("Dia_")
grouped_results_df = grouped_results_df.query("target == 'HFD'")
grouped_results_df

In [ ]:
best_models_df = grouped_results_df.groupby(["cv_split", "attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])

In [ ]:
best_models_df

In [ ]:
rename_cols = {"Sensitivity (Recall)": "Sensitivity", "AUC": "c-index", "target": "Predicting", "attr_group": "Attributes"}
final_selection = ["cv_split", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

In [ ]:
#compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].iloc[:-1].rename(columns=rename_cols)
best_models_df["Majority Accuracy"] = best_models_df["n_positive"]/ best_models_df["n_total"]

In [ ]:
compare_attr_df = best_models_df.rename(columns=rename_cols)
compare_attr_df["cv_split"] = ["by ID", "by centre", "by centre + virtual"]
# compare_attr_df["Attributes"] = ["MICE", "Expert Selection", "Expert + Blood", "Secondary"]
compare_attr_df["Majority Accuracy"] = compare_attr_df["n_positive"]/ compare_attr_df["n_total"]
compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")] = compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")].round(3)
#compare_attr_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_attrs.csv"), index=None)
compare_attr_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert_blood", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert_blood.csv"), index=None)
compare_expert_df.loc[:, final_selection]

In [ ]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[[pd.IndexSlice["secondary", "HFD"]], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_secondary.csv"), index=None)
compare_expert_df.loc[:, final_selection]